<a href="https://colab.research.google.com/github/ciepielajan/SDA_SMOG_Wawelski/blob/main/Beata/Linear_Regression_smog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math
import sklearn
import keras
import sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.cluster import KMeans

In [3]:
link = "https://drive.google.com/file/d/1gwPP5TlL0zYHDkSjowfnR0DU8rsj1bTd/view?usp=sharing"
link_id = link.split("/")[-2]
!gdown --id $link_id
!unzip smog1.zip

Downloading...
From: https://drive.google.com/uc?id=1gwPP5TlL0zYHDkSjowfnR0DU8rsj1bTd
To: /content/smog1.zip
2.39MB [00:00, 76.0MB/s]
Archive:  smog1.zip
  inflating: april-2017.csv          
  inflating: august-2017.csv         
  inflating: december-2017.csv       
  inflating: february-2017.csv       
  inflating: january-2017.csv        
  inflating: july-2017.csv           
  inflating: june-2017.csv           
  inflating: march-2017.csv          
  inflating: may-2017.csv            
  inflating: november-2017.csv       
  inflating: october-2017.csv        
  inflating: sensor_locations.csv    
  inflating: september-2017.csv      


In [9]:
jan = pd.read_csv('january-2017.csv')
feb = pd.read_csv('february-2017.csv')
mar = pd.read_csv('march-2017.csv')
apr = pd.read_csv('april-2017.csv')
may = pd.read_csv('may-2017.csv')
jun = pd.read_csv('june-2017.csv')
jul = pd.read_csv('july-2017.csv')
aug = pd.read_csv('august-2017.csv')
sep = pd.read_csv('september-2017.csv')
octo = pd.read_csv('october-2017.csv')
nov = pd.read_csv('november-2017.csv')
dec = pd.read_csv('december-2017.csv')

In [10]:
year = pd.concat([jan, feb, mar, apr, may, jun, jul, aug, sep, octo, nov, dec])

In [11]:
year.shape

(8593, 337)

In [12]:
year.head(3)

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-01-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,68.0,102279.0,115.0,127.0,196.0,-3.0,57.0,102505.0,196.0,222.0,346.0,0.0,49.0,102689.0,124.0,145.0,229.0,2.0,25.0,102379.0,148.0,176.0,277.0,1.0,27.0,102437.0,...,102253.0,184.0,218.0,322.0,0.0,54.0,101672.0,137.0,153.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01T01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,68.0,102228.0,105.0,116.0,180.0,-3.0,56.0,102454.0,187.0,211.0,330.0,0.0,49.0,102635.0,124.0,145.0,230.0,1.0,25.0,102325.0,131.0,156.0,247.0,0.0,27.0,102387.0,...,102201.0,146.0,169.0,250.0,0.0,52.0,101623.0,119.0,132.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01T02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,66.0,102149.0,110.0,121.0,189.0,-4.0,55.0,102375.0,187.0,212.0,333.0,0.0,49.0,102558.0,108.0,125.0,201.0,1.0,24.0,102244.0,108.0,127.0,203.0,1.0,27.0,102310.0,...,102119.0,138.0,160.0,237.0,0.0,49.0,101536.0,97.0,105.0,162.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
sensors = pd.read_csv("sensor_locations.csv")

In [16]:
year["UTC time"] = pd.to_datetime(year["UTC time"], format="%Y-%m-%dT%H:%M:%S")


In [17]:
year.head(3)

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,68.0,102279.0,115.0,127.0,196.0,-3.0,57.0,102505.0,196.0,222.0,346.0,0.0,49.0,102689.0,124.0,145.0,229.0,2.0,25.0,102379.0,148.0,176.0,277.0,1.0,27.0,102437.0,...,102253.0,184.0,218.0,322.0,0.0,54.0,101672.0,137.0,153.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,68.0,102228.0,105.0,116.0,180.0,-3.0,56.0,102454.0,187.0,211.0,330.0,0.0,49.0,102635.0,124.0,145.0,230.0,1.0,25.0,102325.0,131.0,156.0,247.0,0.0,27.0,102387.0,...,102201.0,146.0,169.0,250.0,0.0,52.0,101623.0,119.0,132.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,66.0,102149.0,110.0,121.0,189.0,-4.0,55.0,102375.0,187.0,212.0,333.0,0.0,49.0,102558.0,108.0,125.0,201.0,1.0,24.0,102244.0,108.0,127.0,203.0,1.0,27.0,102310.0,...,102119.0,138.0,160.0,237.0,0.0,49.0,101536.0,97.0,105.0,162.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
year.shape

(8593, 337)

In [19]:
year = year.set_index("UTC time")

In [22]:
year.head(1)

,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,140_pm25,140_pm10,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10,147_temperature,147_humidity,147_pressure,147_pm1,147_pm25,147_pm10,169_temperature,169_humidity,169_pressure,169_pm1,169_pm25,169_pm10,170_temperature,170_humidity,170_pressure,170_pm1,170_pm25,170_pm10,171_temperature,171_humidity,171_pressure,171_pm1,...,228_pressure,228_pm1,228_pm25,228_pm10,263_temperature,263_humidity,263_pressure,263_pm1,263_pm25,263_pm10,622_temperature,622_humidity,622_pressure,622_pm1,622_pm25,622_pm10,713_temperature,713_humidity,713_pressure,713_pm1,713_pm25,713_pm10,808_temperature,808_humidity,808_pressure,808_pm1,808_pm25,808_pm10,857_temperature,857_humidity,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
UTC time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,68.0,102279.0,115.0,127.0,196.0,-3.0,57.0,102505.0,196.0,222.0,346.0,0.0,49.0,102689.0,124.0,145.0,229.0,2.0,25.0,102379.0,148.0,176.0,277.0,1.0,27.0,102437.0,147.0,...,102253.0,184.0,218.0,322.0,0.0,54.0,101672.0,137.0,153.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
df2= year.groupby(year.columns.str.split('_').str[0], axis=1)

In [80]:
new_df = pd.DataFrame()

In [81]:
for i in df2:
  old = i[1].columns
  new = i[1].columns.str.split("_").str[1]
  zmiana = { k:v  for k,v in zip(old,new)}
  i[1].rename(columns=zmiana,inplace=True)
  i[1]["id"] = i[0]
  new_df = new_df.append(i[1])

In [82]:
new_df

,temperature,humidity,pressure,pm1,pm25,pm10,id
UTC time,,,,,,,
2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2017-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
...,...,...,...,...,...,...,...
2017-12-24 20:00:00,5.0,85.0,101798.0,13.0,15.0,26.0,895
2017-12-24 21:00:00,5.0,85.0,101840.0,12.0,13.0,23.0,895
2017-12-24 22:00:00,5.0,82.0,101839.0,10.0,12.0,21.0,895


In [105]:
new_df["id"] = new_df["id"].astype(int)
sensors["id"] = sensors["id"].astype(int)

In [106]:
new_df = new_df.reset_index()
new_df.head()

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id
0,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
3,2017-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140
4,2017-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140


In [107]:
geo_df = new_df.merge(sensors, how='left', on='id')
geo_df

,UTC time,temperature,humidity,pressure,pm1,pm25,pm10,id,latitude,longitude
0,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
3,2017-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
4,2017-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,140,50.057747,19.961374
...,...,...,...,...,...,...,...,...,...,...
481203,2017-12-24 20:00:00,5.0,85.0,101798.0,13.0,15.0,26.0,895,50.084966,19.877998
481204,2017-12-24 21:00:00,5.0,85.0,101840.0,12.0,13.0,23.0,895,50.084966,19.877998
481205,2017-12-24 22:00:00,5.0,82.0,101839.0,10.0,12.0,21.0,895,50.084966,19.877998
481206,2017-12-24 23:00:00,5.0,85.0,101837.0,10.0,12.0,21.0,895,50.084966,19.877998


In [141]:
from numpy import nan
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# PUNKT 7
# 7.1 REGRESJA LINIOWA

In [ ]:
# PIERWSZA PROBA MODELOWANIA REGRESJA LINIOWA.
# BEZ USUWANIA NAN, BEZ USREDNIANIA NAN, BEZ UWZGLEDNIANIA CZASU. TYLKO MONITORING I WSPOLRZEDNE DO PREDYKCJI

In [142]:
geo_df = geo_df.replace({nan:0})

In [143]:
X = geo_df[['temperature', 'humidity', 'pressure', 'latitude', 'longitude' ]]
y = geo_df['pm10']


In [144]:
X = X.values

In [145]:
y = y.values

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=42)

In [148]:


linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [149]:
y_pred = linear_regression.predict(X_test)

In [150]:
y_pred

array([ 36.27827405,  64.94619771,  32.67470927, ..., -11.00975079,
       -11.29744028,  28.43380237])

In [152]:
id=1
linear_regression_prediction = linear_regression.predict(X_test[id,:].reshape(1,-1))

In [153]:
print("Model predicted for  {0} value {1}".format(id, linear_regression_prediction))

Model predicted for  1 value [64.94619771]


In [154]:
print("Real value for  \"{0}\" is {1}".format(id, y_test[id]))

Real value for  "1" is 25.0


In [ ]:
# WNIOSKI: MODEL POMYLIL SIE O PONAD 50%

In [155]:
print('Coefficients of a learned model: \n', linear_regression.coef_)

Coefficients of a learned model: 
 [-2.25229025e+00  6.10121588e-03  3.42436750e-04 -3.38278297e+01
  4.03833159e+01]


In [156]:
linear_regression_predictions = linear_regression.predict(X_test)
print(linear_regression_predictions)

[ 36.27827405  64.94619771  32.67470927 ... -11.00975079 -11.29744028
  28.43380237]


In [ ]:
# EWALUACJA MODELU

In [159]:
from sklearn.metrics import mean_squared_error
print("Mean squared error of a learned model: %.2f" % 
      mean_squared_error(y_test, linear_regression.predict(X_test)))

Mean squared error of a learned model: 2270.00


In [160]:
from sklearn.metrics import r2_score
print('Variance score: %.2f' % r2_score(y_test, linear_regression.predict(X_test)))

Variance score: 0.18


In [161]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LinearRegression(), X, y, cv=4)
print(scores)

[  0.11402681 -22.65645845   0.25334255   0.16230449]


In [ ]:
# TODO
# WYDAJE MI SIE ZE JAK TAKI PROSTY MODEL I NIE WYSZEDL ZA BARDZO TO TRZEBA BY SPROBOWAC COS POLEPSZYC JUZ NA TYM ETAPIE. 1. MOZE USREDNIC NANY, 2.MOZE USUNAC NANY, 3.MOZE ZROBIC GRID SEARCH, 
# A MOZE TO WLASNIE NIE UWZGLEDIENIE ZALEZNOSCI CZASOWYCH POWODUJE TE BLEDY I TO BY SIE WYDAWALO OCZYWISTE ZE MOZE JEDNAK SPORBOWAC PRZEJSC DO ANALIZY SZEREGOW CZASOWYCH
# 

In [ ]:
# SPROBUJE ZASTOSOWAC SDREDNA KROCZACA DO NASZYCH DANYCH. JUTRO DAM WYKRESY DO SZEREGOW CZASOWYCH

In [ ]:
# WYDAJE MI SIE ZE ROBIENIE TEJ SIATKI PUNKTOW ODBYWA SIE POPRZEZ OBLICZANIE SREDNIEJ KROCZACEJ Z WSPOLRZEDNYCH. TYLKO TRZEBA OKRESLIC ROZMIAR KROKU CZYLI ZMIERZYC 
# GOOGLE MAP ILE METROW ODPOWIADA PRZESUNIECIE O 1 STOPIEN I OBLICZENIE O ILE SIE ZMIENIAJA WSPORZEDNE JAK ODSTEP BEDZIE 500M